In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
tf.keras.backend.clear_session()

In [7]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [8]:
subject = 'Paraná - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [9]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_PR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Produção de Cimento (t),Paraná - Consumo de Cimento (t)
0,2008,8.033948,0.782138,0.768548,7.755114e+06,422290.202070,9.424555,6.724366e+06,0.745648,0.834122,...,11.760983,17.558665,912.624708,1226.296848,18.012164,13.653739,78.027482,2.236583,5058.377260,3001.521000
1,2009,8.063370,0.782910,0.767137,7.805166e+06,424173.603186,9.409042,6.722105e+06,0.748601,0.835310,...,9.472585,16.652522,912.466179,1217.653383,17.954863,13.668549,78.238643,2.155189,5346.288081,3092.068000
2,2010,8.026843,0.783594,0.766111,7.991788e+06,416829.675918,9.394384,6.762279e+06,0.751387,0.835940,...,9.332485,17.928528,912.482023,1194.060192,17.703414,13.001948,76.311573,2.092804,5527.794738,3749.448000
3,2011,8.013806,0.784190,0.765458,8.164220e+06,410997.875281,9.386906,6.795421e+06,0.753998,0.836020,...,10.985689,18.661309,912.629463,1182.416452,17.710041,12.630186,74.615878,2.043995,5623.811148,4174.942000
4,2012,6.472496,0.784389,0.764921,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.755977,0.835445,...,8.159014,19.310061,912.778766,1176.491125,17.874291,12.612094,73.865245,1.941425,5779.044240,4522.969000
5,2013,7.011799,0.785421,0.765581,8.462722e+06,403688.428174,9.371494,6.849508e+06,0.759142,0.834565,...,7.913561,19.233593,912.914618,1170.878793,18.105264,12.720105,73.487865,1.844730,5855.685230,4578.716000
6,2014,7.462443,0.785706,0.765963,8.560896e+06,400704.121188,9.343267,6.852462e+06,0.760987,0.833112,...,10.350442,20.725057,912.973266,1150.562966,18.275455,13.157699,74.605015,1.851460,5788.881494,4341.821857
7,2015,7.813261,0.785515,0.766043,8.597116e+06,391942.316578,9.284504,6.825760e+06,0.762152,0.831122,...,12.454398,22.054778,913.057535,1145.712487,18.177724,13.346348,75.170150,1.905966,5749.529291,4108.786286
8,2016,7.840669,0.788366,0.758774,8.620855e+06,389049.090551,9.247193,6.809907e+06,0.771114,0.838386,...,13.111908,22.315526,913.132268,1158.573328,18.115048,13.261122,74.043778,1.933375,5786.537903,3869.334857
9,2017,7.553100,0.787582,0.759026,8.632052e+06,389052.079088,9.235469,6.808242e+06,0.770943,0.835784,...,9.479812,21.977949,913.165670,1160.957791,18.281050,13.470416,73.546507,1.889856,5902.212019,3764.518000


In [10]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Produção de Cimento (t)
0,8.033948,0.782138,0.768548,7.755114e+06,422290.202070,9.424555,6.724366e+06,0.745648,0.834122,5.483335e+08,...,2.868108,11.760983,17.558665,912.624708,1226.296848,18.012164,13.653739,78.027482,2.236583,5058.377260
1,8.063370,0.782910,0.767137,7.805166e+06,424173.603186,9.409042,6.722105e+06,0.748601,0.835310,5.302764e+08,...,2.305885,9.472585,16.652522,912.466179,1217.653383,17.954863,13.668549,78.238643,2.155189,5346.288081
2,8.026843,0.783594,0.766111,7.991788e+06,416829.675918,9.394384,6.762279e+06,0.751387,0.835940,5.046425e+08,...,1.958246,9.332485,17.928528,912.482023,1194.060192,17.703414,13.001948,76.311573,2.092804,5527.794738
3,8.013806,0.784190,0.765458,8.164220e+06,410997.875281,9.386906,6.795421e+06,0.753998,0.836020,4.854622e+08,...,2.079088,10.985689,18.661309,912.629463,1182.416452,17.710041,12.630186,74.615878,2.043995,5623.811148
4,6.472496,0.784389,0.764921,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.755977,0.835445,4.867661e+08,...,2.183282,8.159014,19.310061,912.778766,1176.491125,17.874291,12.612094,73.865245,1.941425,5779.044240
5,7.011799,0.785421,0.765581,8.462722e+06,403688.428174,9.371494,6.849508e+06,0.759142,0.834565,5.225848e+08,...,2.403445,7.913561,19.233593,912.914618,1170.878793,18.105264,12.720105,73.487865,1.844730,5855.685230
6,7.462443,0.785706,0.765963,8.560896e+06,400704.121188,9.343267,6.852462e+06,0.760987,0.833112,5.676413e+08,...,2.748397,10.350442,20.725057,912.973266,1150.562966,18.275455,13.157699,74.605015,1.851460,5788.881494
7,7.813261,0.785515,0.766043,8.597116e+06,391942.316578,9.284504,6.825760e+06,0.762152,0.831122,5.179392e+08,...,2.501529,12.454398,22.054778,913.057535,1145.712487,18.177724,13.346348,75.170150,1.905966,5749.529291
8,7.840669,0.788366,0.758774,8.620855e+06,389049.090551,9.247193,6.809907e+06,0.771114,0.838386,4.524144e+08,...,2.073464,13.111908,22.315526,913.132268,1158.573328,18.115048,13.261122,74.043778,1.933375,5786.537903
9,7.553100,0.787582,0.759026,8.632052e+06,389052.079088,9.235469,6.808242e+06,0.770943,0.835784,4.213689e+08,...,2.177386,9.479812,21.977949,913.165670,1160.957791,18.281050,13.470416,73.546507,1.889856,5902.212019


In [11]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     3092.068000
1     3749.448000
2     4174.942000
3     4522.969000
4     4578.716000
5     4341.821857
6     4108.786286
7     3869.334857
8     3764.518000
9     3773.354000
10    3786.974000
11    4235.532000
12    4784.413000
13    4538.256000
14            NaN
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [12]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Produção de Cimento (t)
0,0.954430,-1.699678,1.331277,-1.852442,1.525597,1.304983,-1.799100,-1.621982,-0.042462,1.204752,...,2.026352,0.984445,-1.266771,-0.975425,1.940409,-0.268956,1.137644,1.835993,2.079287,-1.984438
1,1.017927,-1.261317,0.939993,-1.694326,1.677576,1.094681,-1.854414,-1.293295,0.429908,0.820657,...,0.085558,-0.051710,-1.745665,-1.582298,1.579454,-0.539264,1.176616,1.956737,1.461930,-1.162017
2,0.939098,-0.873574,0.655199,-1.104775,1.084964,0.895963,-0.871808,-0.983091,0.680031,0.275392,...,-1.114490,-0.115145,-1.071300,-1.521642,0.594192,-1.725446,-0.577589,0.854816,0.988761,-0.643541
3,0.910963,-0.535386,0.474312,-0.560051,0.614372,0.794584,-0.061186,-0.692424,0.712066,-0.132596,...,-0.697344,0.633403,-0.684027,-0.957222,0.107944,-1.694185,-1.555909,-0.114802,0.618558,-0.369269
4,-2.415320,-0.422527,0.325283,-0.050154,0.319763,0.747334,0.727969,-0.472052,0.483303,-0.104860,...,-0.337667,-0.646476,-0.341164,-0.385670,-0.139500,-0.919352,-1.603519,-0.544022,-0.159410,0.074157
5,-1.251457,0.163252,0.508405,0.382937,0.024542,0.585657,1.261692,-0.119710,0.133711,0.657046,...,0.422337,-0.757614,-0.381577,0.134391,-0.373874,0.170234,-1.319280,-0.759813,-0.892823,0.293083
6,-0.278927,0.325010,0.614316,0.693077,-0.216274,0.203003,1.333947,0.085716,-0.443735,1.615453,...,1.613108,0.345772,0.406656,0.358905,-1.222272,0.973093,-0.167721,-0.121014,-0.841773,0.102257
7,0.478169,0.216545,0.636582,0.807496,-0.923301,-0.593627,0.680859,0.215483,-1.234902,0.558230,...,0.760923,1.298413,1.109410,0.681498,-1.424830,0.512055,0.328724,0.202137,-0.428361,-0.010153
8,0.537319,1.834498,-1.380170,0.882491,-1.156767,-1.099436,0.293116,1.213246,1.652199,-0.835563,...,-0.716758,1.596124,1.247214,0.967586,-0.887756,0.216390,0.104443,-0.441935,-0.220470,0.095563
9,-0.083280,1.389158,-1.310222,0.917863,-1.156526,-1.258364,0.252392,1.194161,0.618336,-1.495939,...,-0.358019,-0.048438,1.068806,1.095455,-0.788179,0.999483,0.655217,-0.726280,-0.550555,0.425987


In [13]:
reshaped_train_input = dfToInputRNN(train_input)

In [14]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[-0.42881683,  0.05023865, -1.29053308,  0.69018756,
         -0.77950283, -1.23123715, -0.10251621,  1.11506099,
         -1.3313138 , -1.0453709 ,  1.21859064, -0.99976018,
          0.79428371,  1.15575749, -0.52559858,  1.30260559,
         -0.55253046, -0.25130359,  1.38787176,  1.47154451,
         -1.80649616,  1.09296291,  0.94281138,  0.52836084,
          0.61258103,  0.87656257,  1.47801536,  1.71982086,
          1.28221181,  1.39086472,  1.58209844,  1.08872091,
          0.55212962, -1.81398118,  1.44113674,  1.11191094,
          1.30470835,  0.86390516, -1.67365968, -1.70738939,
         -1.55813196, -0.48498301, -0.75714639,  0.40770893,
         -0.27394734,  2.02323219,  1.2908951 ,  1.70465513,
          1.81148084,  1.39968653, -1.36191442,  1.28024593,
          1.11054795,  0.12662799,  1.40414112,  1.40468999,
          1.45643959,  1.58509336,  1.32283157,  1.30222047,
          1.55138602,  1.91801602, -0.77489784,  2.32098884,
         -1.1942541 , -1

In [15]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     3092.068000
1     3749.448000
2     4174.942000
3     4522.969000
4     4578.716000
5     4341.821857
6     4108.786286
7     3869.334857
8     3764.518000
9     3773.354000
10    3786.974000
11    4235.532000
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [16]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    4235.532
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [17]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Produção de Cimento (t)
12,-0.734266,-1.069894,-1.42466,0.401023,-0.568106,-1.099064,-0.414155,1.035451,-2.637193,1.108405,...,-1.482779,-2.260447,0.964616,0.339314,-0.540552,0.224215,0.839814,-2.237536,-1.155068,2.508234


In [18]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-0.73426567, -1.06989407, -1.42465965,  0.40102323,
         -0.56810629, -1.099064  , -0.41415465,  1.03545073,
         -2.63719315,  1.10840491,  2.63582309, -0.18995911,
          0.72640489,  2.66833308, -0.032632  ,  0.98802435,
          0.24980494, -0.08955808,  1.85434394,  2.02706568,
         -2.21150703,  1.42938591,  0.85132668,  0.14092386,
          0.30902324,  1.00267904,  2.27970137,  1.11272593,
          0.61007902,  0.89064538,  1.42231438,  1.02650562,
          1.47127593, -1.53201473,  1.73503858,  1.66610613,
          1.0456894 ,  0.55289259, -1.81216593, -2.10342296,
         -2.55377478,  0.41940628, -1.75756445,  0.241023  ,
         -0.98659345,  1.52184916,  1.01398607,  1.49290967,
          1.40764092,  1.87930411, -2.51892072,  2.83356214,
          1.58620924, -0.02524128,  1.54231542,  1.5176119 ,
          1.7355269 ,  1.87941982,  1.54198129,  1.51276712,
          1.83795594,  2.62210693,  1.04993174,  0.96476493,
         -1.48277861, -2

In [19]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    4784.413
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [20]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [23]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [24]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2621823830, 366725844, 1592710421, 1178241486, 32521976, 3207316778, 931388765, 3433303843, 1315639157, 2952255976, 2389680722, 2296758710, 1500888134, 4067333373, 3518395123, 1423705443, 1313927141, 2630427549, 3598139054, 1801053331, 152276515, 2693006889, 1696263430, 1784948758, 2454914437, 1306543466, 2158202413, 4226526201, 492410543, 3625762954, 2154009309, 3607166980, 2736956266, 3422878446, 771213929, 1902108384, 913101629, 2287334500, 4191304255, 2765544422, 4276513894, 342449106, 2922142802, 1930539143, 575332906, 674417103, 336865672, 3355486962, 199587058, 2904146146, 2901997310, 3135763770, 4200005027, 3673473126, 1504164770, 3760221627, 2684510310, 2110397170, 663435458, 137902719, 1860223867, 2502670886, 4151802263, 341607733, 3830842676, 449880611, 134799611, 15397067, 1614583885, 183189638, 741725495, 2129944895, 2892967796, 571612965, 3109954742, 2171174934, 2395420659, 2549966559, 3241211796, 2346974677, 2593578163, 1812064895, 3139099870, 4292182272, 910922372, 148

loss: 34054.52734375


Step: 93 ___________________________________________
loss: 40110.62109375


Step: 94 ___________________________________________
loss: 33615.328125


Step: 95 ___________________________________________
loss: 56772.71875


Step: 96 ___________________________________________
loss: 60555.98828125


Step: 97 ___________________________________________
loss: 47929.61328125


Step: 98 ___________________________________________
loss: 56271.52734375


Step: 99 ___________________________________________
loss: 31568.037109375


final_seed: 2684510310


In [25]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 16155485.0000
Epoch 2/10000
1/1 [==============================] - 0s 5ms/step - loss: 15465901.0000
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 5375486.5000
Epoch 4/10000
1/1 [==============================] - 0s 5ms/step - loss: 1595142.3750
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 1392449.6250
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 1174711.1250
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 1044754.5000
Epoch 8/10000
1/1 [==============================] - 0s 5ms/step - loss: 249950.2031
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 248417.8594
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 260318.3125
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 368962.2500
Epoch 12/10000
1/1 [==============================] - 0

1/1 [==============================] - 0s 4ms/step - loss: 154644.9219
Epoch 97/10000
1/1 [==============================] - 0s 4ms/step - loss: 158003.5781
Epoch 98/10000
1/1 [==============================] - 0s 5ms/step - loss: 283232.9688
Epoch 99/10000
1/1 [==============================] - 0s 5ms/step - loss: 169710.7656
Epoch 100/10000
1/1 [==============================] - 0s 5ms/step - loss: 212947.1719
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 96045.1250
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 131817.6250
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 194546.3281
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 236804.1719
Epoch 105/10000
1/1 [==============================] - 0s 5ms/step - loss: 204594.6094
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 107344.2109
Epoch 107/10000
1/1 [==============================] - 0s 4ms/s

1/1 [==============================] - 0s 5ms/step - loss: 206745.7969
Epoch 191/10000
1/1 [==============================] - 0s 4ms/step - loss: 194635.8281
Epoch 192/10000
1/1 [==============================] - 0s 4ms/step - loss: 126982.3047
Epoch 193/10000
1/1 [==============================] - 0s 5ms/step - loss: 131914.1406
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 154929.3750
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 68386.2656
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 129640.6875
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 295418.3438
Epoch 198/10000
1/1 [==============================] - 0s 5ms/step - loss: 255876.5469
Epoch 199/10000
1/1 [==============================] - 0s 5ms/step - loss: 177496.6250
Epoch 200/10000
1/1 [==============================] - 0s 5ms/step - loss: 53327.8945
Epoch 201/10000
1/1 [==============================] - 0s 4ms

1/1 [==============================] - 0s 4ms/step - loss: 201706.4375
Epoch 285/10000
1/1 [==============================] - 0s 4ms/step - loss: 125680.3516
Epoch 286/10000
1/1 [==============================] - 0s 4ms/step - loss: 175794.7500
Epoch 287/10000
1/1 [==============================] - 0s 5ms/step - loss: 161640.5000
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 150119.0469
Epoch 289/10000
1/1 [==============================] - 0s 5ms/step - loss: 215128.7500
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 161761.5625
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 245587.9219
Epoch 292/10000
1/1 [==============================] - 0s 5ms/step - loss: 210791.2344
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 157922.4531
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 194273.5781
Epoch 295/10000
1/1 [==============================] - 0s 5

1/1 [==============================] - 0s 4ms/step - loss: 248212.8750
Epoch 379/10000
1/1 [==============================] - 0s 4ms/step - loss: 203141.0781
Epoch 380/10000
1/1 [==============================] - 0s 4ms/step - loss: 136943.4219
Epoch 381/10000
1/1 [==============================] - 0s 5ms/step - loss: 275603.1562
Epoch 382/10000
1/1 [==============================] - 0s 4ms/step - loss: 172563.8750
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 195996.7031
Epoch 384/10000
1/1 [==============================] - 0s 5ms/step - loss: 169444.9219
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 294803.2812
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 212174.8594
Epoch 387/10000
1/1 [==============================] - 0s 5ms/step - loss: 236765.5469
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 282621.9062
Epoch 389/10000
1/1 [==============================] - 0s 4

...

1/1 [==============================] - 0s 5ms/step - loss: 141716.9219
Epoch 567/10000
1/1 [==============================] - 0s 4ms/step - loss: 239234.7031
Epoch 568/10000
1/1 [==============================] - 0s 5ms/step - loss: 243458.1406
Epoch 569/10000
1/1 [==============================] - 0s 4ms/step - loss: 171775.7031
Epoch 570/10000
1/1 [==============================] - 0s 4ms/step - loss: 290427.0312
Epoch 571/10000
1/1 [==============================] - 0s 5ms/step - loss: 281032.5312
Epoch 572/10000
1/1 [==============================] - 0s 4ms/step - loss: 114585.5859
Epoch 573/10000
1/1 [==============================] - 0s 4ms/step - loss: 157885.1719
Epoch 574/10000
1/1 [==============================] - 0s 5ms/step - loss: 77691.5781
Epoch 575/10000
1/1 [==============================] - 0s 4ms/step - loss: 139066.2969
Epoch 576/10000
1/1 [==============================] - 0s 4ms/step - loss: 253144.2656
Epoch 577/10000
1/1 [==============================] - 0s 5m

1/1 [==============================] - 0s 4ms/step - loss: 105626.4766
Epoch 661/10000
1/1 [==============================] - 0s 4ms/step - loss: 242253.7031
Epoch 662/10000
1/1 [==============================] - 0s 4ms/step - loss: 225853.6719
Epoch 663/10000
1/1 [==============================] - 0s 4ms/step - loss: 172443.2656
Epoch 664/10000
1/1 [==============================] - 0s 4ms/step - loss: 182152.0469
Epoch 665/10000
1/1 [==============================] - 0s 4ms/step - loss: 197327.1719
Epoch 666/10000
1/1 [==============================] - 0s 4ms/step - loss: 156126.4375
Epoch 667/10000
1/1 [==============================] - 0s 4ms/step - loss: 188053.6875
Epoch 668/10000
1/1 [==============================] - 0s 4ms/step - loss: 157342.7344
Epoch 669/10000
1/1 [==============================] - 0s 4ms/step - loss: 126844.3516
Epoch 670/10000
1/1 [==============================] - 0s 4ms/step - loss: 311141.3125
Epoch 671/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 5ms/step - loss: 224051.3750
Epoch 755/10000
1/1 [==============================] - 0s 4ms/step - loss: 133369.3906
Epoch 756/10000
1/1 [==============================] - 0s 4ms/step - loss: 146070.3750
Epoch 757/10000
1/1 [==============================] - 0s 4ms/step - loss: 245873.3281
Epoch 758/10000
1/1 [==============================] - 0s 4ms/step - loss: 92694.1875
Epoch 759/10000
1/1 [==============================] - 0s 4ms/step - loss: 206215.9375
Epoch 760/10000
1/1 [==============================] - 0s 4ms/step - loss: 108512.2734
Epoch 761/10000
1/1 [==============================] - 0s 4ms/step - loss: 79716.9297
Epoch 762/10000
1/1 [==============================] - 0s 4ms/step - loss: 229455.6719
Epoch 763/10000
1/1 [==============================] - 0s 4ms/step - loss: 146550.7031
Epoch 764/10000
1/1 [==============================] - 0s 4ms/step - loss: 180564.9219
Epoch 765/10000
1/1 [==============================] - 0s 4ms

1/1 [==============================] - 0s 4ms/step - loss: 104697.0391
Epoch 849/10000
1/1 [==============================] - 0s 4ms/step - loss: 155071.1875
Epoch 850/10000
1/1 [==============================] - 0s 4ms/step - loss: 106018.8125
Epoch 851/10000
1/1 [==============================] - 0s 4ms/step - loss: 202165.5469
Epoch 852/10000
1/1 [==============================] - 0s 4ms/step - loss: 282699.4062
Epoch 853/10000
1/1 [==============================] - 0s 4ms/step - loss: 128151.4141
Epoch 854/10000
1/1 [==============================] - 0s 4ms/step - loss: 111358.3359
Epoch 855/10000
1/1 [==============================] - 0s 4ms/step - loss: 236809.2344
Epoch 856/10000
1/1 [==============================] - 0s 4ms/step - loss: 225649.8281
Epoch 857/10000
1/1 [==============================] - 0s 4ms/step - loss: 75572.1953
Epoch 858/10000
1/1 [==============================] - 0s 4ms/step - loss: 200243.6875
Epoch 859/10000
1/1 [==============================] - 0s 4m

1/1 [==============================] - 0s 4ms/step - loss: 137041.3281
Epoch 943/10000
1/1 [==============================] - 0s 4ms/step - loss: 134164.3125
Epoch 944/10000
1/1 [==============================] - 0s 4ms/step - loss: 185880.8906
Epoch 945/10000
1/1 [==============================] - 0s 4ms/step - loss: 160537.7031
Epoch 946/10000
1/1 [==============================] - 0s 4ms/step - loss: 224589.5000
Epoch 947/10000
1/1 [==============================] - 0s 4ms/step - loss: 213330.0625
Epoch 948/10000
1/1 [==============================] - 0s 4ms/step - loss: 103710.1875
Epoch 949/10000
1/1 [==============================] - 0s 4ms/step - loss: 157809.4844
Epoch 950/10000
1/1 [==============================] - 0s 5ms/step - loss: 129527.9609
Epoch 951/10000
1/1 [==============================] - 0s 4ms/step - loss: 149036.1875
Epoch 952/10000
1/1 [==============================] - 0s 4ms/step - loss: 220045.7969
Epoch 953/10000
1/1 [==============================] - 0s 4

In [26]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 617ms/step
[4784.413](test_target) - [3907.852](prediction) = 876.5609492187496


In [27]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.1832118065933584

In [28]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [29]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Produção de Cimento (t)
0,-0.470460,-1.248978,1.283752,-0.938459,0.382853,1.236143,-0.644859,-1.236383,-1.329522,1.148205,...,1.308949,1.412346,0.333480,1.408380,1.000190,0.909918,0.683149,0.581225,1.275014,-1.306146
1,1.390219,0.049998,-0.128080,-0.446997,0.987585,-0.023123,-0.767579,0.023618,0.247295,0.140893,...,-0.190804,-0.643250,-1.356947,-0.815318,0.365766,0.482610,0.730797,0.825914,-0.107644,0.183513
2,-0.919759,1.198980,-1.155672,1.385456,-1.370438,-1.213019,1.412438,1.212765,1.082228,-1.289098,...,-1.118144,-0.769096,1.023467,-0.593062,-1.365956,-1.392529,-1.413946,-1.407140,-1.167370,1.122633


train_target:


,Paraná - Consumo de Cimento (t)
0,3092.068
1,3749.448
2,4174.942


test_input:


...

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Produção de Cimento (t)
12,-0.734266,-1.069894,-1.42466,0.401023,-0.568106,-1.099064,-0.414155,1.035451,-2.637193,1.108405,...,-1.482779,-2.260447,0.964616,0.339314,-0.540552,0.224215,0.839814,-2.237536,-1.155068,2.508234


test_target:


,Paraná - Consumo de Cimento (t)
12,4784.413


1/1 [==============================] - 1s 537ms/step
Error: 876.5609492187496




[4206.767578125,
 4468.83740234375,
 3366.90380859375,
 4039.09765625,
 4066.13818359375,
 3846.37744140625,
 3879.57958984375,
 3777.4658203125,
 3811.4541015625,
 3907.85205078125]

In [30]:
display(targets)
display(predictions)

[4522.969,
 4578.716,
 4341.821857142857,
 4108.786285714286,
 3869.334857142857,
 3764.518,
 3773.354,
 3786.974,
 4235.532,
 4784.413]

[4206.767578125,
 4468.83740234375,
 3366.90380859375,
 4039.09765625,
 4066.13818359375,
 3846.37744140625,
 3879.57958984375,
 3777.4658203125,
 3811.4541015625,
 3907.85205078125]

In [31]:
mae = mean_absolute_error(predictions, targets)
mae

316.5722082589287

In [32]:
porcentage = mae/np.mean(targets)
porcentage

0.07579587042377961